In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

In [2]:
model_name = "Qwen/QwQ-32B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
prompt = "How many r's are in the word \"strawberry\""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
model_inputs

{'input_ids': tensor([[151644,    872,    198,   4340,   1657,    435,    594,    525,    304,
            279,   3409,    330,    495,    672,  15357,      1, 151645,    198,
         151644,  77091,    198, 151667,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [4]:
start = time.time()
n_tokens = 10
generated_ids = model.generate(
    **model_inputs,
    # max_new_tokens=32768
    max_new_tokens=n_tokens
)
time_diff = time.time() - start
if time_diff < n_tokens:
    print(f"The generation speed is {n_tokens/time_diff:.2f} tokens/sec")
else:
    print(f"The generation speed is {time_diff/n_tokens:.2f} sec/iter")

The generation speed is 3.34 sec/iter


In [5]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Okay, so I need to figure out how many
